In [9]:
import torch
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams["figure.figsize"] = 16, 6

In [10]:
def plot_sample(model, inputs, image_dims=21):
    """
    Plot a reconstruced image from variational auotencoder.

    Parameters:
    ----------
    model : Pytorch nn.Module
        - Variational autoencoder model.

    inputs : Pytorch tensor
        - Input tensor for the variational autoencoder
    """
    # Getting started...
    plt.imshow(model(inputs).data[0].numpy().reshape(image_dims, image_dims), cmap='inferno')

In [11]:
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable


class Normal(object):
    def __init__(self, mu, sigma, log_sigma, v=None, r=None):
        self.mu = mu
        self.sigma = sigma  # either stdev diagonal itself, or stdev diagonal from decomposition
        self.logsigma = log_sigma
        dim = mu.get_shape()
        if v is None:
            v = torch.FloatTensor(*dim)
        if r is None:
            r = torch.FloatTensor(*dim)
        self.v = v
        self.r = r


def latent_loss(z_mean, z_stddev):
    mean_sq = z_mean * z_mean
    stddev_sq = z_stddev * z_stddev
    return 0.5 * torch.mean(mean_sq + stddev_sq - torch.log(stddev_sq) - 1)


class VAE(torch.nn.Module):
    latent_dim = 8

    def __init__(self, encoder, decoder, z_mean=0.0, z_sigma=0.0, use_cuda=True):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.use_cuda = use_cuda
        self.z_mean = z_mean
        self.z_sigma = z_sigma

    def _sample_latent(self, h_enc):
        """
        Return the latent normal sample z ~ N(mu, sigma^2)
        """
        mu = h_enc
        log_sigma = h_enc
        sigma = torch.exp(log_sigma)
        std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()
        if self.use_cuda:
            std_z = std_z.cuda()

        self.z_mean = mu
        self.z_sigma = sigma

        # Reparameterization trick
        return mu + sigma * Variable(std_z, requires_grad=False)

    def forward(self, state):
        h_enc = self.encoder(state)
        #print('h_enc is of type {}'.format(type(h_enc)))
        z = self._sample_latent(h_enc)
        #print('z has shape {}'.format(z.shape))
        out = self.decoder(z)
        #print('output has shape {}'.format(out.shape))
        return out

In [12]:
class Decoder(nn.Module):
    def __init__(self, latent_size, output_size, kernel1=4, stride1=2, kernel2=4,
                 stride2=2, kernel3=4, stride3=2, kernel4=4, stride4=2):
        super(Decoder, self).__init__()
        """
        Parameters:
        ----------
        latent_size : int
            latent dimension size of the autoencoder.
        output_size : int
            Output dimension for the data. Should equal input_dimension of AE.
        kernel* : int, defualt=4
            Convolutional filter size for layer *.
        stride* : int, default=2
            Stride length for convolutional filter at layer *.
        """
        self.latent_size = latent_size
        self.output_size = output_size
        self.kernel1 = kernel1
        self.stride1 = stride1
        self.kernel2 = kernel2
        self.stride2 = stride2
        self.kernel3 = kernel3
        self.stride3 = stride3
        self.kernel4 = kernel4
        self.stride4 = stride4

        self.fc = nn.Linear(self.latent_size, 256)

        self.cnn_decoder1 = nn.Sequential(
            nn.ConvTranspose2d(1, 16, self.kernel1, self.stride1, padding=2),
            nn.AdaptiveMaxPool2d(16),
            nn.ELU()
        )

        self.cnn_decoder2 = nn.Sequential(
            nn.ConvTranspose2d(16, 16, self.kernel2, self.stride2),
            nn.AdaptiveMaxPool2d(16),
            nn.ELU()
        )

        self.cnn_decoder3 = nn.Sequential(
            nn.ConvTranspose2d(16, 32, self.kernel3, self.stride3),
            nn.AdaptiveMaxPool2d(32),
            nn.ELU()
        )
        
        self.cnn_decoder4 = nn.Sequential(
            nn.ConvTranspose2d(32, 32, self.kernel4, self.stride4),
            nn.AdaptiveMaxPool2d(2),
            nn.ELU()
        )

        self.fc2 = nn.Linear(128, self.output_size)

    def forward(self, latent_input):
        """
        Parameters:
        ----------
        input : float tensor shape=(batch_size, input_size)

        Returns:
        -------
        A float tensor with shape (batch_size, output_size)
        """
        out = self.fc(latent_input)
        out = out.view(out.size(0), 1, 16, 16)
        out = self.cnn_decoder1(out)
        out = self.cnn_decoder2(out)
        out = self.cnn_decoder3(out)
        out = self.cnn_decoder4(out)
        #print('output size of decoder 4 {}'.format(out.size()))
        out = out.view(out.size(0), -1)
        out = self.fc2(out)
        return out

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_size, latent_size, kernel1=4, stride1=1, kernel2=4,
                 stride2=1, kernel3=4, stride3=1, kernel4=4, stride4=1):
        super(Encoder, self).__init__()
        """
        Parameters:
        ----------
        input_size : int
            Input dimension for the data.
        latent_size : int
            Latent space dimension
        kernel* : int, defualt=4
            Convolutional filter size for layer *.
        stride* : int, default=1
            Stride length for convolutional filter at layer *.
        """
        self.input_size = input_size
        self.latent_size = latent_size
        self.kernel1 = kernel1
        self.stride1 = stride1
        self.kernel2 = kernel2
        self.stride2 = stride2
        self.kernel3 = kernel3
        self.stride3 = stride3
        self.kernel4 = kernel4
        self.stride4 = stride4

        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(1, 16, self.kernel1, self.stride1, padding=2),
            nn.AdaptiveMaxPool2d(16),
            nn.ELU(),

            nn.Conv2d(16, 16, self.kernel2, self.stride2, padding=2),
            nn.AdaptiveMaxPool2d(16),
            nn.ELU(),

            nn.Conv2d(16, 32, self.kernel3, self.stride3, padding=2),
            nn.AdaptiveMaxPool2d(32),
            nn.ELU(),

            nn.Conv2d(32, 32, self.kernel4, self.stride4, padding=2),
            nn.AdaptiveMaxPool2d(2),
            nn.ELU()
        )

        self.fc = nn.Linear(128, 8) # 64*2*2
        

    def forward(self, input):
        """
        Parameters:
        ----------
        input : float tensor shape=(batch_size, input_size)

        Returns:
        -------
        A float tensor with shape (batch_size, latent_variable_size)
        """
        out = self.cnn_encoder(input)
        #print('output size of encoder: {}'.format(out.size()))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [14]:
from torch.utils.data import DataLoader

import os
import torch
import numpy as np
from torch.utils.data import Dataset


class FSPeptide(Dataset):
    """
    FS-Peptide Dataset.
    """
    def __init__(self, data, labels, transform=None):
        """
        Parameters:
        ----------
        data : str
            Path to the data in numpy format.
        label : str
            Path to the labels.
        transform : callable, optional
            Optional transform to be applied on a sample.
        """
        self.data = np.load(data)
        self.labels = np.load(labels)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        cont_mat = self.data[index]
        cont_label = self.labels[index]
        sample = {'cont_matrix': cont_mat, 'label': cont_label}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [22]:
batch_size = 1
use_cuda = False

train_data = FSPeptide(data='/Users/youngtodd/data/fspeptide/train.npy',
                       labels='/Users/youngtodd/data/fspeptide/y_train.npy')

trainloader = DataLoader(train_data, batch_size=batch_size)

In [25]:
# Contact matrices are 21x21
input_size = 441 

encoder = Encoder(input_size=input_size, latent_size=8)
decoder = Decoder(latent_size=8, output_size=input_size)
vae = VAE(encoder, decoder, use_cuda=use_cuda)

In [24]:
save_point = '/Users/youngtodd/molecules/molecules/variational_autoencoder/model_saves/epoch10'

vae.load_state_dict(torch.load(save_point))

AssertionError: Torch not compiled with CUDA enabled

In [20]:
for batch_idx, data in enumerate(trainloader):
    inputs = data['cont_matrix']
    inputs = inputs.resize_(batch_size, 1, 21, 21)
    inputs = inputs.float()

    if batch_idx == 10:
        break